In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
DIR1 = '/content/drive/MyDrive/(22-1)캡스톤/recomm/data/preprocessed/data1'
DIR2 = '/content/drive/MyDrive/(22-1)캡스톤/recomm/data/preprocessed/data2'
DIR3 = '/content/drive/MyDrive/(22-1)캡스톤/recomm/data/preprocessed/data3'
DIR4 = '/content/drive/MyDrive/(22-1)캡스톤/recomm/data/preprocessed/data4'
DIR5 = '/content/drive/MyDrive/(22-1)캡스톤/recomm/data/preprocessed/data5'

In [85]:
N_PROBLEM = 24830

In [219]:
import os
import numpy as np
import pandas as pd
import ast
from scipy import sparse
import math
import tensorflow as tf

1. 데이터 로드
- 사용자 ID --> 클러스터 check --> 푼 문제들 csv 파일에 반영 --> matrix 생성
2. Latent Factor CF
- 계산
- 높은 결과 순으로 n개 뽑아냄
- *변경을 어떻게 행렬에 반영?*
- *baseline 코드를 sparse matrix를 사용할 수 있게 변경*  
3. 난이도 필터링
4. 결과

# 1. 데이터 로드

In [203]:
def load_csv(now):
  train = pd.read_csv(os.path.join(DIR1, now+'_train.csv'))
  valid_X = pd.read_csv(os.path.join(DIR1, now+'_validation_X.csv'))
  valid_y = pd.read_csv(os.path.join(DIR1, now+'_validation_y.csv'))
  test_X = pd.read_csv(os.path.join(DIR1, now+'_test_X.csv'))
  test_y = pd.read_csv(os.path.join(DIR1, now+'_test_y.csv'))
  handles = list(set(list(train['handle'])+list(valid_X['handle'])+list(test_X['handle'])))
  return handles, train, valid_X, valid_y, test_X, test_y

In [204]:
def handle2idx(handles, handle):
  return handle.apply(lambda x: handles.index(x))

In [214]:
def load_matrix(data, handles):

    users_no = len(handles)+1
    problems_no = N_PROBLEM + 1

    data = data.dropna(axis=0)
    rows = handle2idx(handles, data['handle'])

    data['IdwithLevel'] = data['IdwithLevel'].apply(lambda x: ast.literal_eval(x))
    problems = data['IdwithLevel'].apply(lambda x: [i for i in x.keys()][0])
    cols = problems
    
    mat = sparse.csr_matrix((np.ones_like(rows),
                             (rows, cols)), dtype='float64',
                             shape=(users_no, problems_no))
    return mat

In [215]:
def load_data(handles, train, valid_X, valid_y, test_X, test_y):
  train = load_matrix(train, handles)
  valid_X = load_matrix(valid_X, handles)
  valid_y = load_matrix(valid_y, handles)
  test_X = load_matrix(test_X, handles)
  test_y = load_matrix(test_y, handles)
  return train, valid_X, valid_y, test_X, test_y

In [216]:
d1_handles, d1_train, d1_valid_X, d1_valid_y, d1_test_X, d1_test_y = load_csv('d1')

In [218]:
d1_train, d1_valid_X, d1_valid_y, d1_test_X, d1_test_y = load_data(d1_handles, d1_train, d1_valid_X, d1_valid_y, d1_test_X, d1_test_y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


# 2. Latent-Factor CF

In [ ]:
class GMF(tf.keras.Model):
  def __init__(self, users_no, problems_no, epochs=200, K=10, lam=1e-2, lr=1e-2, batch_size= 200, random_seed=None):
    # users_no: user 수
    # problems_no: 문제 수
    # K: 잠재요인 개수
    # lr: learning rate
    # lam: regularzation 계수
    # P, Q: 분해된 행렬
    super(LF_CF, self).__init__()
    self.user_no = users_no
    self.problems = problems_no
    self.epochs = epochs
    self.K = K
    self.lam = lam
    self.lr = lr
    self.random_seed = random_seed
    self.batch_size = 200
    self.P = np.random.normal(scale=1./K, size=(users_no, K)) # 정규분포 값으로 초기화
    self.Q = np.random.normal(scale=1./K, size=(problems_no, K))

  # forward pass
  def call(self, X, training=True):

    

In [ ]:
for epoch in range(epochs):
    ################## 학습 ############################
    for step, start in enumerate(range(0, users_no, batch_size)):
        end = min(start + batch_size, users_no)
        X = d1_train[np.arange(start,end+1)]
        
        if sparse.isspmatrix(X):
            X = X.toarray()
        X = X.astype('float32')

        # 1. 행렬 분해
        MF(X)
    rmse = RMSE(X)
  
    ################### 검증 ###########################
    for step, start in enumerate(range(0, users_no, batch_size)):
        end = min(start + batch_size, users_no)
        X_valid = d1_valid_X[np.arange(start,end+1)]
        
        if sparse.isspmatrix(X_valid):
            X_valid = X_valid.toarray()
        X_valid = X_valid.astype('float32')

        # 1. 행렬 분해
        MF(X_valid, False)
    pred_mat = np.dot(P, Q.T)

    # 2. 난이도 필터링
    result = sort_by_level()

    # 3. 평가지표 출력
    recall = RECALL(result, d1_valid_y, batch_size = 200)
    hitrate = RECALL(result, d1_valid_y, batch_size = 200)


In [ ]:
def matrix_factorization(R, K, steps=200, lr=1e-2, r_lambda=1e-2):

  prev_rmse = 10000
  break_count = 0

  # R > 0인 행 위치, 열 위치, 값을 non_zeros list에 저장
  non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j]>0]

  # SGD 기법으로 P, Q 업데이트
  for step in range(steps):
    for i,j,r in non_zeros:
      eij = r - np.dot(P[i,:], Q[j,:].T)

      P[i,:] = P[i,:] + lr*(eij*Q[j,:] - r_lambda*P[i,:])
      Q[j,:] = Q[j,:] + lr*(eij*P[i,:] - r_lambda*Q[j,:])

    rmse = get_rmse(R,P,Q,non_zeros)
    if step % 10 == 0:
      print("### iteration step: ", step, " rmse: ", rmse)
  
  return P, Q

In [ ]:
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, lr=1e-2, r_lambda=1e-2)